# Computer Vision & Internet of Things

#### The Sparks Foundation

## Task - Detection of face mask
#### *By- Kamad Saxena*

In this task we were supposed to develop a model that can detect whether any person is wearing a face mask or not. For achieving this goal, I have used Tensorflow object detection model and used a package called labelImg to label the images in the dataset and train the model using those labelled images.

## Dependencies  


In [1]:
import os
# model
import tensorflow as tf
# vizualization
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import cv2
import numpy as np

## Setting up the paths

In [2]:
WORKPLACE_PATH = 'Tensorflow/workplace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKPLACE_PATH + '/annotations'
IMAGE_PATH = WORKPLACE_PATH + '/images'
MODEL_PATH = WORKPLACE_PATH + '/models'
PRETRAINED_MODEL_PATH = WORKPLACE_PATH + '/pre-trained-models'
CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/'

CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

## Generating Labels

There are only 2 labels that the model has to recognise named below:

In [3]:
labels = [
    {'name':'mask', 'id':1},
    {'name':'no mask', 'id':2},
]

In [ ]:
with open(os.path.join(ANNOTATION_PATH, 'label_map.pbtxt'), 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')
print('Generated label map')

## Generating TF records

Before training our model we have to generate train and test records

In [ ]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

## Downloading TF models

In [ ]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

In [ ]:
!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

## Setting up pipeline config

Once we are done with downloading the model, we have to set up the pipeline configuration of the model acoording to our requirements

In [ ]:
!mkdir {MODEL_PATH + CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}

**model.ssd.num_classes** = 2

**train_config.batch_size** = 4

**train_config.fine_tune_checkpoint** = 'Tensorflow/workplace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'

**train_config.fine_tune_checkpoint_type** = "detection"

**train_input_reader.label_map_path** = ANNOTATION_PATH + '/label_map.pbtxt'

**train_input_reader.tf_record_input_reader.input_path[:]** = [ANNOTATION_PATH + '/train.record']

**eval_input_reader[0].label_map_path** = ANNOTATION_PATH + '/label_map.pbtxt'

**eval_input_reader[0].tf_record_input_reader.input_path[:]** = [ANNOTATION_PATH + '/test.record']

## Training the model

In this step we have to run the command given below in the command prompt in order to train our model. Here, I am training the model for 50000 steps

In [4]:
print(f"""python {APIMODEL_PATH}/research/object_detection/model_main_tf2.py --model_dir={MODEL_PATH}/{CUSTOM_MODEL_NAME} --pipeline_config_path={MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config --num_train_steps=50000""")

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workplace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workplace/models/my_ssd_mobnet/pipeline.config --num_train_steps=50000


## Vizualization

Once we are done with the model training, we are now gonna work on the visualisation part

There are various checkpoints created while training and we need them to test the working of the model, here I am using `ckpt-51`

In [3]:
num = int(input("Enter latest checkpoint: "))

Enter latest checkpoint: 51


Now we will use tensorflow's object detection model and here we are loading it

In [4]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model = detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, f'ckpt-{num}')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [5]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH + '/label_map.pbtxt')

With the help of opencv, we are now gonna test our model in real time 

In [10]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [11]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes'] + label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw = 1,
                min_score_thresh = .3,
                agnostic_mode = False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### Accuracy
With mask: 98%
Without mask: 100%